In [0]:
## https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
def download_data(a,d):  
  import requests
  def download_file_from_google_drive(id, destination):
      URL = "https://docs.google.com/uc?export=download"

      session = requests.Session()

      response = session.get(URL, params = { 'id' : id }, stream = True)
      token = get_confirm_token(response)

      if token:
          params = { 'id' : id, 'confirm' : token }
          response = session.get(URL, params = params, stream = True)

      save_response_content(response, destination)    

  def get_confirm_token(response):
      for key, value in response.cookies.items():
          if key.startswith('download_warning'):
              return value

      return None

  def save_response_content(response, destination):
      CHUNK_SIZE = 32768

      with open(destination, "wb") as f:
          for chunk in response.iter_content(CHUNK_SIZE):
              if chunk: # filter out keep-alive new chunks
                  f.write(chunk)
  download_file_from_google_drive(a,d)
  

In [0]:
# The implementation of all the algorithms have been referenced from scikit-learn documentation.
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2 

a = '1dv2x819EC6xozGCL5iniAD_9h2HIMGDF'
d = 'business.csv'
download_data(a,d)

df = pd.read_csv("business.csv")
X = df.iloc[:,4:]
y = df.iloc[:,3]
y = y.astype('int')

In [4]:
# X = SelectKBest(chi2, k=200).fit_transform(X, y)
# y = y.values.reshape(y.shape[0],1)
# print(X.shape)
# print(y.shape)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#Multiple linear regression
mse = []
mse_train = []
for i in range(100,400,100):
  pca = PCA(n_components=i)
  X1 = pca.fit_transform(X)
  X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.3, random_state=42)
  from sklearn.linear_model import LinearRegression
  reg = LinearRegression().fit(X_train, y_train)
  y_pred = reg.predict(X_train)
  mse_train.append(mean_squared_error(y_train,y_pred))

  y_pred = reg.predict(X_test)
  mse.append(mean_squared_error(y_test,y_pred))
print("The ideal no. of features: ",(mse.index(min(mse))+1)*100) 
print('==== Training:')
print("The mean sqaured error is :",min(mse_train))
print("Root Mean Squared error: ",np.sqrt(min(mse_train)))
print('\n==== Testing:')
print("The mean sqaured error is :",min(mse))
print("Root Mean Squared error: ",np.sqrt(min(mse)))

The ideal no. of features:  300
==== Training:
The mean sqaured error is : 0.5609269807807401
Root Mean Squared error:  0.7489505863411418

==== Testing:
The mean sqaured error is : 0.5772762042950652
Root Mean Squared error:  0.7597869466469303


In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components=300)
X = pca.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
#Decision Tree Regressor
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(random_state=42).fit(X_train, y_train)

print("Decision Tree Regressor: ")
print('==== Training:')
y_pred = dt.predict(X_train)
print("Mean Squared error: ",mean_squared_error(y_train,y_pred))
print("Root Mean Squared error: ",np.sqrt(mean_squared_error(y_train,y_pred)))
print('\n==== Testing:')
y_pred = dt.predict(X_test)
print("Mean Squared error: ",mean_squared_error(y_test,y_pred))
print("Root Mean Squared error: ",np.sqrt(mean_squared_error(y_test,y_pred)))

Decision Tree Regressor: 
==== Training:
Mean Squared error:  0.0351960212835919
Root Mean Squared error:  0.18760602677843774

==== Testing:
Mean Squared error:  1.0939107941683528
Root Mean Squared error:  1.0459019046585358


In [7]:
#Random Forest
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=30, random_state=42).fit(X_train, y_train)
# y_pred = rf.predict(X_test)
print("RandomForestRegressor: ")
print('==== Training:')
y_pred = rf.predict(X_train)
print("Mean Squared error: ",mean_squared_error(y_train,y_pred))
print("Root Mean Squared error: ",np.sqrt(mean_squared_error(y_train,y_pred)))
print('\n==== Testing:')
y_pred = rf.predict(X_test)
print("Mean Squared error: ",mean_squared_error(y_test,y_pred))
print("Root Mean Squared error: ",np.sqrt(mean_squared_error(y_test,y_pred)))

RandomForestRegressor: 
==== Training:
Mean Squared error:  0.11435813051254834
Root Mean Squared error:  0.33816878997410205

==== Testing:
Mean Squared error:  0.6240074032288073
Root Mean Squared error:  0.7899413922746468


In [8]:
#K-nearest neighbours
from sklearn.neighbors import KNeighborsRegressor
k = KNeighborsRegressor(n_neighbors=5).fit(X, y) 
# y_pred = k.predict(X_test)
print("Knearest neighbours Regressor: ")
print('==== Training:')
y_pred = k.predict(X_train)
print("Mean Squared error: ",mean_squared_error(y_train,y_pred))
print("Root Mean Squared error: ",np.sqrt(mean_squared_error(y_train,y_pred)))
print('\n==== Testing:')
y_pred = k.predict(X_test)
print("Mean Squared error: ",mean_squared_error(y_test,y_pred))
print("Root Mean Squared error: ",np.sqrt(mean_squared_error(y_test,y_pred)))

Knearest neighbours Regressor: 
==== Training:
Mean Squared error:  0.4431042059008161
Root Mean Squared error:  0.6656607288257406

==== Testing:
Mean Squared error:  0.45502746246796055
Root Mean Squared error:  0.6745572343900557


In [9]:
#Neural Network
#The implementation of below algorithm has been referenced from keras documentation
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from keras import Sequential
from keras.layers import Dense,Dropout
num_hidden_layers=3
regressor = Sequential()

regressor.add(Dense(128, activation='relu', kernel_initializer='random_normal', input_dim=X_train.shape[1]))
regressor.add(Dropout(0.2))
for i in range(num_hidden_layers-1):
  regressor.add(Dense(128, activation='relu', kernel_initializer='random_normal'))
  regressor.add(Dropout(0.2))

regressor.add(Dense(1, activation='linear', kernel_initializer='random_normal'))

regressor.compile(optimizer='adam',loss='mean_squared_error')
regressor.fit(X_train,y_train, epochs=20,verbose=0)
y_pred = regressor.predict(X_test)
print("Neural Networks: ")
print('==== Training:')
y_pred = regressor.predict(X_train)
print("Mean Squared error: ",mean_squared_error(y_train,y_pred))
print("Root Mean Squared error: ",np.sqrt(mean_squared_error(y_train,y_pred)))
print('\n==== Testing:')
y_pred = regressor.predict(X_test)
print("Mean Squared error: ",mean_squared_error(y_test,y_pred))
print("Root Mean Squared error: ",np.sqrt(mean_squared_error(y_test,y_pred)))

Using TensorFlow backend.


Neural Networks: 
==== Training:
Mean Squared error:  0.5221397628062207
Root Mean Squared error:  0.7225923904984197

==== Testing:
Mean Squared error:  0.5644458839334424
Root Mean Squared error:  0.7512961359766483
